In [1]:
# 필요한 패키지 불러오기
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler

# 데이터 로드
train_df = pd.read_csv('train_for_NLP.csv')
test_df = pd.read_csv('test_cleaned.csv')

# train 데이터에서 특성과 레이블 분리
X_train = train_df['combined_str']
y_train = train_df['target']
X_test = test_df['combined_str']

# Word2Vec 모델 학습 (Skipgram 사용)
sentences = [text.split() for text in X_train]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, sg=1, workers=4)

# Word2Vec으로 문서 벡터화
def vectorize_w2v(text, model):
    words = text.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    
    # 단어 벡터가 없는 경우를 대비해 예외 처리 (np.zeros로 채움)
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(100)  # Word2Vec 벡터 크기(100차원)

# 문장을 Word2Vec으로 벡터화
X_train_w2v = np.array([vectorize_w2v(text, w2v_model) for text in X_train])
X_test_w2v = np.array([vectorize_w2v(text, w2v_model) for text in X_test])

# 스케일링
scaler = StandardScaler()
X_train_w2v_scaled = scaler.fit_transform(X_train_w2v)
X_test_w2v_scaled = scaler.transform(X_test_w2v)

# Gaussian Naive Bayes 모델 학습
gnb_model = GaussianNB()
gnb_model.fit(X_train_w2v_scaled, y_train)

# 테스트 데이터 예측
y_test_pred = gnb_model.predict(X_test_w2v_scaled)

# 결과 출력
submission_df = pd.read_csv("sample_submission.csv")
submission_df['target'] = y_test_pred
submission_df.to_csv("submit_Word2Vec_NaiveBayes.csv", index=False)
